### Build and push docker image from ECR 
* Get aws account related variables such as account id and region and compose the full docker image name 
* Dockerfile takes the fullname as a parameter to build docker image 
* Push to ecr

In [ ]:
%%bash
./build-and-push.sh

* changed to your own image uri

In [ ]:
%env fullname=230755935769.dkr.ecr.us-east-1.amazonaws.com/yolo-tensorflow

In [ ]:
!echo $fullname 

In [ ]:
!pygmentize build-and-push.sh

In [ ]:
!pygmentize container/Dockerfile

In [ ]:
!pygmentize container/main.py

#### training data file structure 
* files under cfg - data descriptors and model config 
* files under dinfo - paths to training images 
* files under train2014 - image files and labels 

In [ ]:
!wget -O demo.zip "https://tinyurl.com/y3tx7nh9"
!unzip demo.zip 
!mv sagemaker_training_job /home/ec2-user/sagemaker_training_job

In [ ]:
import os 
def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))
            
list_files('/home/ec2-user/sagemaker_training_job/')            

In [ ]:
!cat /home/ec2-user/sagemaker_training_job/config_path/config.json

In [ ]:
!cat /home/ec2-user/sagemaker_training_job/dinfo/obj.data 

In [ ]:
!cat /home/ec2-user/sagemaker_training_job/dinfo/train.txt

In [ ]:
!cat /home/ec2-user/sagemaker_training_job/cfg/yolov4.cfg 

#### Yolo V4 

In [ ]:
from IPython.display import Image
Image('/home/ec2-user/sagemaker_training_job/COCO_train2014_000000387731.jpg')

In [ ]:
!nvidia-docker run -it -v /home/ec2-user/sagemaker_training_job:/opt/ml/input/data \
    $fullname \
    predict_local \
    /opt/ml/input/data/cfg/yolov4.cfg \
    /opt/ml/input/data/dinfo/coco.data \
    /opt/ml/input/data/yolo_model/yolov4.weights \
    /opt/ml/input/data/COCO_train2014_000000387731.jpg \
    /opt/ml/input/data/    

In [ ]:
Image('/home/ec2-user/sagemaker_training_job/predictions.jpg')

In [ ]:
Image('/home/ec2-user/sagemaker_training_job/BloodImage_00203_jpg.rf.7e1e0baedaf6227f45f74ed5727ccec8.jpg')

In [ ]:
!nvidia-docker run -it -v /home/ec2-user/sagemaker_training_job:/opt/ml/input/data \
    $fullname \
    predict_local \
    /opt/ml/input/data/cfg/yolov4.cfg \
    /opt/ml/input/data/dinfo/coco.data \
    /opt/ml/input/data/yolo_model/yolov4.weights \
    /opt/ml/input/data/BloodImage_00203_jpg.rf.7e1e0baedaf6227f45f74ed5727ccec8.jpg \
    /opt/ml/input/data

In [ ]:
Image('/home/ec2-user/sagemaker_training_job/predictions.jpg')

#### Transfer Yolo V4 model into blood cells recognition 

In [ ]:
!nvidia-docker run -it -v /home/ec2-user/sagemaker_training_job:/opt/ml/input/data \
    $fullname \
    train


In [ ]:
!nvidia-docker run -it -v /home/ec2-user/sagemaker_training_job:/opt/ml/input/data \
    $fullname \
    predict_local \
    /opt/ml/input/data/cfg/custom-yolov4-detector.cfg \
    /opt/ml/input/data/dinfo/obj.data \
    /opt/ml/input/data/last_backup/custom-yolov4-detector_final.weights \
    /opt/ml/input/data/BloodImage_00203_jpg.rf.7e1e0baedaf6227f45f74ed5727ccec8.jpg \
    /opt/ml/input/data

In [ ]:
from IPython.display import Image
Image('/home/ec2-user/sagemaker_training_job/predictions.jpg')

* create your own s3 bucket! 

In [ ]:
!aws s3 mb s3://ws-yolov4-yianc/ --region us-east-1

In [ ]:
!aws s3 cp --recursive /home/ec2-user/sagemaker_training_job s3://ws-yolov4-yianc/sagemaker_training_job

In [ ]:
import boto3
region = boto3.session.Session().region_name
bucket = 'ws-yolov4-yianc'


In [ ]:
account=boto3.client('sts').get_caller_identity().get('Account')
repositoryUri="{}.dkr.ecr.{}.amazonaws.com/yolo-tensorflow".format(account, region)
repositoryUri

In [ ]:
from datetime import datetime
now = datetime.now()
job_name = 'yolov4-' + now.strftime("%Y-%m-%d-%H-%M-%S")
job_name

In [ ]:
import sagemaker
from sagemaker import get_execution_role
role = get_execution_role()

In [ ]:
cfg='s3://{}/sagemaker_training_job/cfg/'.format(bucket)
dinfo='s3://{}/sagemaker_training_job/dinfo/'.format(bucket)
yolo_model='s3://{}/sagemaker_training_job/yolo_model/'.format(bucket)
train='s3://{}/sagemaker_training_job/train/'.format(bucket)
config_path='s3://{}/sagemaker_training_job/config_path/'.format(bucket)
outpath='s3://{}/model/'.format(bucket)
(cfg, dinfo, train, config_path, yolo_model, outpath) 


In [ ]:
sm = boto3.client('sagemaker')

response = sm.create_training_job(
      TrainingJobName=job_name,
      HyperParameters={'momentum':'0.8'},
      AlgorithmSpecification={
          'TrainingImage': repositoryUri,
          'TrainingInputMode': 'File',
          "MetricDefinitions" : [
            {
            "Name": "IOU",
            "Regex": "\(IOU: (.*?),"
            },
             {
            "Name": "GIOU",
            "Regex": "GIOU: (.*?)\)"
            }]
      },
      RoleArn=role,
      InputDataConfig=[
          {
              'ChannelName': 'cfg',
              'DataSource': {
                  'S3DataSource': {
                      'S3DataType': 'S3Prefix',
                      'S3Uri': cfg,
                      'S3DataDistributionType': 'FullyReplicated',
                  },
              },
              'InputMode': 'File'
          },
          {
              'ChannelName': 'train',
              'DataSource': {
                  'S3DataSource': {
                      'S3DataType': 'S3Prefix',                      
                      'S3Uri': train,
                      'S3DataDistributionType': 'FullyReplicated',
                  },
              },
              'InputMode': 'File'
          },
          {
              'ChannelName': 'config_path',
              'DataSource': {
                  'S3DataSource': {
                      'S3DataType': 'S3Prefix',                      
                      'S3Uri': config_path,
                      'S3DataDistributionType': 'FullyReplicated',
                  },
              },
              'InputMode': 'File'
          },
          {
              'ChannelName': 'dinfo',
              'DataSource': {
                  'S3DataSource': {
                      'S3DataType': 'S3Prefix',                      
                      'S3Uri': dinfo,
                      'S3DataDistributionType': 'FullyReplicated',
                  },
              },
              'InputMode': 'File'
          },
          {
              'ChannelName': 'yolo_model',
              'DataSource': {
                  'S3DataSource': {
                      'S3DataType': 'S3Prefix',                      
                      'S3Uri': yolo_model,
                      'S3DataDistributionType': 'FullyReplicated',
                  },
              },
              'InputMode': 'File'
          },
      ],
      OutputDataConfig={
          'S3OutputPath': outpath
      },
      ResourceConfig={
          'InstanceType': 'ml.p3.2xlarge',
          'InstanceCount': 1,
          'VolumeSizeInGB': 10,
      },
      StoppingCondition={
        'MaxRuntimeInSeconds': 60*60*5,
      }
  )
response